## 1. Installing and Loading Packages

In [1]:
# %pip install selenium
# %pip install webdriver-manager
# %pip install selenium-stealth
# %pip install undetected-chromedriver
# %pip install fake-useragent
# %pip install free-proxy

In [2]:
import pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
from pprint import pprint
import io
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
import random
from selenium_stealth import stealth
import undetected_chromedriver as uc
from fake_useragent import UserAgent
import pandas as pd
import csv
import os

## 2. Web Scraping Tripadvisor Reviews

### Setting up selenium + chrome drivers
- prevent bot detection by: 
    1. cycling through list of user agents

In [29]:
def get_free_proxies(driver):
    driver.get('https://sslproxies.org')
    headers = [header.text.strip() for header in driver.find_elements(By.XPATH, '//table/thead/tr/th')]
    rows = driver.find_elements(By.XPATH, '//table/tbody/tr')
    
    proxies = []
    for row in rows:
        proxy_data = dict(zip(headers, [td.text.strip() for td in row.find_elements(By.TAG_NAME, 'td')]))
        proxies.append(proxy_data)
        
    driver.quit()
    return proxies

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

free_proxies = get_free_proxies(driver)
print("Number of free proxies retrieved:", len(free_proxies))
print("5 sample proxy data:")
pprint(free_proxies[:5])

Number of free proxies retrieved: 116
5 sample proxy data:
[{'Anonymity': 'anonymous',
  'Code': 'SG',
  'Country': 'Singapore',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '8.219.97.248',
  'Last Checked': '15 secs ago',
  'Port': '80'},
 {'Anonymity': 'anonymous',
  'Code': 'IT',
  'Country': 'Italy',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '34.154.161.152',
  'Last Checked': '15 secs ago',
  'Port': '80'},
 {'Anonymity': 'anonymous',
  'Code': 'US',
  'Country': 'United States',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '5.161.191.226',
  'Last Checked': '15 secs ago',
  'Port': '3128'},
 {'Anonymity': 'anonymous',
  'Code': 'SG',
  'Country': 'Singapore',
  'Google': 'no',
  'Https': 'yes',
  'IP Address': '20.6.0.172',
  'Last Checked': '10 mins ago',
  'Port': '80'},
 {'Anonymity': 'anonymous',
  'Code': 'BR',
  'Country': 'Brazil',
  'Google': 'yes',
  'Https': 'yes',
  'IP Address': '200.174.198.95',
  'Last Checked': '10 mins ago',
  'Port': '8888

In [30]:
user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/98.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/98.0.0.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/609.1.15',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Safari/609.1.15',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/98.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Edge/18.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Safari/609.1.15',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/97.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/97.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/97.0.0.0',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/17.0.0.0',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Edge/17.0.0.0',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/17.0.0.0',
]

In [164]:
# url = 'https://www.tripadvisor.com/Airlines'
# service = ChromeService(ChromeDriverManager().install())
# user_agent = random.choice(user_agents)
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument(f'user-agent={user_agent}')
# chrome_options.add_argument('--disable-blink-features=AutomationControlled')
# chrome_options.add_argument("start-maximized")
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
# chrome_options.add_experimental_option('useAutomationExtension', False)
# driver = webdriver.Chrome(service=service, options=chrome_options)
# driver.get(url)
# time.sleep(3)
# pickle.dump(driver.get_cookies(), open('cookies.pkl', 'wb'))

In [26]:
def setup_webdriver():
    while True:
        user_agent = random.choice(user_agents)
        print(f"Using user agent: {user_agent}")
        
        proxy_choice = random.choice(free_proxies)
        proxy_ip_port = f"{proxy_choice['IP Address']}:{proxy_choice['Port']}"
        print(f"Using proxy: {proxy_ip_port}")

        service = Service(ChromeDriverManager().install())
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument(f'user-agent={user_agent}')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_argument("start-maximized")
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        chrome_options.add_argument(f'--proxy-server={proxy_ip_port}')

        try:
            driver = webdriver.Chrome(service=service, options=chrome_options)
            time.sleep(1)
            driver.quit()
            return driver
        except Exception as e:
            print(f"Error setting up WebDriver: {e}")
            continue

### List of airline reviews in tripadvisor

In [63]:
def scrape_page(driver, csv_writer):
    try:
        airlines = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".prw_rup.prw_airlines_airline_lander_card"))
        )
        for airline in airlines:
            name = airline.find_element(By.CSS_SELECTOR, ".airlineName").text
            review_link_element = airline.find_element(By.CSS_SELECTOR, "a.review_button.ui_button.secondary.small")
            review_link = review_link_element.get_attribute("href")
            num_reviews = airline.find_element(By.CSS_SELECTOR, ".airlineReviews").text
            csv_writer.writerow([name, review_link, num_reviews])

    except (NoSuchElementException, TimeoutException):
        print("Error occurred while scraping.")

In [64]:
def main1():
    driver = setup_webdriver()
    driver.get("https://www.tripadvisor.com/Airlines")
    
    with open('scraped_data.csv', 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Airline Name', 'Review Link', 'Number of Reviews'])

        page_number = 1
        while True:
            print(f"Scraping page {page_number}...")
            scrape_page(driver, csv_writer)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "span.nav.next.ui_button.primary"))
                )
                if "disabled" in next_button.get_attribute("class"):
                    print("Reached the last page.")
                    break

                next_button.click()
                WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
                page_number += 1
                time.sleep(3)

            except (NoSuchElementException, TimeoutException):
                print("Reached the end of the pages or encountered an error.")
                break

    driver.quit()

In [47]:
main1()

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Error occurred while scraping.
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Error occurred while scraping.
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Error occurred while scraping.
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Error occurred while scraping.
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Error occurred while scraping.
Scraping page 41...
Scraping page 42...
Error occurred

### ⚠️ Gather each airline reviews [v3]

#### Creating the basic scraping functions

In [132]:
import json

def extract_proxy_addresses(file_path):
    proxies = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                try:
                    proxy_data = json.loads(line.strip())["proxies"]
                    for proxy in proxy_data:
                        ip_address = proxy["ip"]
                        port = proxy["port"]
                        proxies.append(f"{ip_address}:{port}")
                except json.JSONDecodeError:
                    print(f"Invalid JSON in line: {line.strip()}")
                    continue
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return proxies


file_path = './http_socks5_socks4_proxies.txt'
proxy_list = [proxy for proxy in extract_proxy_addresses(file_path)]

In [ ]:
def init_driver():
    ua = UserAgent()
    my_user_agent = ua.chrome

    # for proxy in proxies:
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument(f"user-agent={my_user_agent}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
        # options.add_argument(f'--proxy-server={proxy}')

    # try:
    service = Service(ChromeDriverManager().install())
    driver = uc.Chrome(service=service, options=options)

    # driver.get("http://example.com")
    # if driver.title:
    #     print(f"Proxy {proxy} works. Continuing with this proxy.")
    #     return driver
    # else:
    #     print(f"Proxy {proxy} failed. Trying next.")
    #     driver.quit()
    # except WebDriverException:
    #     print(f"Error with proxy {proxy}. Trying next.")

    print("No working proxies found.")
    return None

# # Example usage
# driver = init_driver(proxy_list)
# if driver:
#     # Do something with the driver
#     pass
# else:
#     print("Failed to initialize driver with proxies.")

In [ ]:
def scrape_review(div):
    
    try:
        reviewer_name = div.find_element(By.CSS_SELECTOR, 'a.ui_header_link.uyyBf').text.strip()
        review_date = div.find_element(By.CSS_SELECTOR, 'div.cRVSd').text.strip()
        review_title = div.find_element(By.CSS_SELECTOR, 'div.KgQgP.MC._S.b.S6.H5._a a.Qwuub').text.strip()
        spans = div.find_elements(By.CSS_SELECTOR, 'span.QewHA.H4._a > span')
        review_text = ' '.join(span.text.strip() for span in spans)
        travel_date = div.find_element(By.CSS_SELECTOR, 'span.teHYY._R.Me.S4.H3').text.strip()

        rating = div.find_element(By.CSS_SELECTOR, 'span.ui_bubble_rating').get_attribute('class')
        overall_rating = int(rating.split('_')[-1]) / 10

        review = {
            'Reviewer Name': reviewer_name,
            'Review Date': review_date,
            'Scraped Date': date.today().strftime('%Y-%m-%d'),
            'Review Title': review_title,
            'Review Text': review_text,
            'Travel Date': travel_date,
            'Overall Rating': overall_rating
        }
        
        categories = ["Legroom", "Seat comfort", "In-flight Entertainment", "Customer service", "Value for money", "Cleanliness", "Check-in and boarding", "Food and Beverage"]

        categories_container = div.find_elements(By.CSS_SELECTOR, 'div.hemdC.S2.H2')
        print(categories_container)

        for category_element in categories_container:
            category_name_span = category_element.find_element(By.CSS_SELECTOR, 'span:nth-last-child(1)') 
            category_name = category_name_span.text.strip()
            print(category_name)

            if category_name in categories:
                rating_span = category_element.find_element(By.CSS_SELECTOR, 'span.ui_bubble_rating')
                rating_class = rating_span.get_attribute('class')
                print(rating_class)
                review[category_name] = int(rating_class.split('_')[-1]) // 10 
                
        flight_details = div.find_elements(By.CSS_SELECTOR, 'div.tNvbV.f div.dmRSR.n.R2.S2')
        if flight_details:
            flight_route = flight_details[0].text if len(flight_details) > 0 else None
            flight_type = flight_details[1].text if len(flight_details) > 1 else None
            flight_class = flight_details[2].text if len(flight_details) > 2 else None

            review['Flight Route'] = flight_route
            review['Flight Type'] = flight_type
            review['Flight Class'] = flight_class
            
        print(review)

        return review
    except Exception as e:
        return None

In [ ]:
def page_has_loaded(driver):
    return driver.execute_script("return document.readyState") == "complete"

def go_to_next_page(driver):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.ui_button.nav.next.primary"))
        )
        
        if "disabled" in next_button.get_attribute("class"):
            print("log: Reached the last page.")
            return False
        
        next_button.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        WebDriverWait(driver, 10).until(page_has_loaded)

        return True

    except (NoSuchElementException, TimeoutException):
        print("log: Reached the end of the pages or encountered an error.")
        return False

In [ ]:
def scrape_tripadvisor_reviews(url):
    for proxy in proxy_list:
        driver = init_driver() 

        if not driver:
            continue

        try:
            driver.get(url)
            print("Connected to the page with proxy:", proxy)
            time.sleep(3)

            reviews = []

            while True:
                review_divs = driver.find_elements(By.CSS_SELECTOR, 'div.lgfjP.Gi.z.pBVnE.MD.bZHZM')
                for div in review_divs:
                    review = scrape_review(div)
                    if review:
                        reviews.append(review) 

                if not go_to_next_page(driver):
                    break

            driver.quit()
            pprint(reviews)
            return reviews

        except (WebDriverException, TimeoutException):
            driver.close()
            driver.quit()
            print(f"Connection failed with proxy {proxy}. Trying next.")

    print("Failed to connect with all proxies.")
    return []

In [ ]:
url = "https://www.tripadvisor.com/Airline_Review-d11831129-Reviews-Aerolinea-De-Antioquia"
res = scrape_tripadvisor_reviews(url)
pprint(res)

In [163]:
options = webdriver.ChromeOptions()
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--incognito')
# options.add_argument(f"user-agent={my_user_agent}")
# options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--no-sandbox")

url = "https://www.tripadvisor.com/Airline_Review-d11831129-Reviews-Aerolinea-De-Antioquia"
driver.get(url)

MaxRetryError: HTTPConnectionPool(host='localhost', port=59019): Max retries exceeded with url: /session/2668899781cd145d9bbf061ff3dd7b81/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x16a652a90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [157]:
ua = UserAgent()
my_user_agent = ua.chrome
my_user_agent

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

#### Processing as a batch

In [177]:
def save_checkpoint(index, checkpoint_file='checkpoint.txt'):
    with open(checkpoint_file, 'w') as file:
        file.write(str(index))

def load_checkpoint(checkpoint_file='checkpoint.txt'):
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as file:
            return int(file.read().strip())
    return 0

def process_batch(urls):
    all_reviews = []
    for url in urls:
        reviews = scrape_tripadvisor_reviews(url)
        all_reviews.extend(reviews)
    return all_reviews

def write_to_csv(reviews, filename):
    if not reviews:
        return

    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=reviews[0].keys())
        if file.tell() == 0:
            writer.writeheader()
        writer.writerows(reviews)

In [178]:
batch_size = 1
filename = 'tripadvisor_reviews.csv'
checkpoint_file = 'checkpoint.txt'

df = pd.read_csv('./scraped_data.csv')

start_index = load_checkpoint(checkpoint_file)
for i in range(start_index, len(df['Review Link']), batch_size):
    batch_urls = df['Review Link'][i:i + batch_size].tolist()
    batch_reviews = process_batch(batch_urls)
    write_to_csv(batch_reviews, filename)
    save_checkpoint(i + batch_size, checkpoint_file)

print("Data collection complete. Check the CSV file.")

log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: Reached the end of the pages or encountered an error.
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: No data found or There is an Error
log: Reached the end of the pages or encountered an error.


KeyboardInterrupt: 

### Gather each airline reviews [v4]

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime

def initialize_webdriver():
    print("started to init")
    options = webdriver.ChromeOptions()
    ua = UserAgent()
    my_user_agent = ua.chrome
    options.add_argument('--ignore-certificate-errors')
    options.add_argument(f"user-agent={my_user_agent}")
    options.add_argument('--incognito')
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=options)
    print("init complete")
    return driver

def click_read_more(driver):
    read_more_buttons = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "div.lszDU div.TnInx[data-test-target='expand-review']")
        )
    )

    for button in read_more_buttons:
        try:
            button.click()
            time.sleep(4)
            print("button clicked !!!")
            WebDriverWait(driver, 10).until(EC.staleness_of(button))
        except Exception as e:
            print("Warning: Button Click Error")
            
def random_mouse_movement(driver):
    action = ActionChains(driver)
    action.move_by_offset(random.randint(0, 100), random.randint(0, 100))
    action.perform()
            
def scrape_webpage(driver, url):
    print("started scraping")
    driver.get(url)
    all_reviews = []
    output_file = 'scraped_airline_reviews.csv'
    
    while True:
        click_read_more(driver)

        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lgfjP.Gi.z.pBVnE.MD.bZHZM")))
        elements = driver.find_elements(By.CLASS_NAME, "lgfjP.Gi.z.pBVnE.MD.bZHZM")
        elements_html = [element.get_attribute('outerHTML') for element in elements]

        all_reviews.extend([parse_review(html, url) for html in elements_html if html is not None])
        # save_reviews_to_csv(all_reviews, output_file)

        # Check for the Next button
        try:
            next_button = driver.find_element(By.CLASS_NAME, "nav.next.primary")
            if 'disabled' in next_button.get_attribute('class'):
                break  # If the Next button is disabled, exit the loop
            next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
            print("Navigated to next page")
        except NoSuchElementException:
            print("No more pages found.")
            break

        # Random mouse movement and sleep to mimic human interaction
        random_mouse_movement(driver)
        random_sleep(2, 5)

    print("scraping done")
    return all_reviews


existing_reviews = set()

def parse_review(html, url):
    soup = BeautifulSoup(html, 'html.parser')
    review_data = {}
    output_file = 'scraped_airline_reviews.csv'
    
    review_data['airline_url'] = url
    user_info = soup.find('a', class_='ui_header_link')
    if user_info:
        user_name = user_info.get_text(strip=True)
        review_date_div = soup.find('div', class_='cRVSd')
        if review_date_div:
            review_date = review_date_div.get_text()
            unique_id = (url, user_name, review_date)
            if unique_id in existing_reviews:
                return None
            existing_reviews.add(unique_id)
            review_data['user_name'] = user_name
            review_data['review_date'] = review_date
            review_data['scraped_date'] = datetime.today().strftime('%Y-%m-%d')
        else:
            return None
    else:
        return None

    travel_details = soup.find_all('div', class_='dmRSR n R2 S2')
    review_data['travel_details'] = [detail.get_text(strip=True) for detail in travel_details]

    review_title = soup.find('div', class_='KgQgP MC _S b S6 H5 _a')
    review_data['review_title'] = review_title.get_text(strip=True) if review_title else None
    review_text = soup.find('span', class_='QewHA H4 _a')
    review_data['review_text'] = review_text.get_text(strip=True) if review_text else None
    
    overall_rating = soup.find('span', class_='ui_bubble_rating')
    review_data['overall_rating'] = int(overall_rating['class'][1].split('_')[-1]) // 10 if overall_rating else None

    subratings = soup.find_all('div', class_='hemdC S2 H2')
    review_data['subratings'] = {}
    for subrating in subratings:
        category = subrating.get_text(strip=True)
        rating_class = subrating.find('span', class_='ui_bubble_rating')['class'][1]
        review_data['subratings'][category] = int(rating_class.split('_')[-1]) // 10
        
    return review_data

def save_reviews_to_csv(reviews, file_path):
    file_exists = os.path.isfile(file_path)
    
    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=reviews[0].keys())
        
        if not file_exists:
            writer.writeheader() 
        
        for review in reviews:
            writer.writerow(review)
            
def check_for_captcha(driver):
    try:
        captcha_iframe = driver.find_element(By.TAG_NAME, "iframe")
        if "captcha" in captcha_iframe.get_attribute("src"):
            return True
    except NoSuchElementException:
        return False

def create_batches(df, batch_size):
    for i in range(0, len(df), batch_size):
        yield df[i:i + batch_size]

def read_processed_links(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return set(line.strip() for line in file)
    return set()

def add_processed_link(file_path, link):
    with open(file_path, 'a') as file:
        file.write(link + '\n')
        
def save_reviews_to_csv(reviews, file_path):
    file_exists = os.path.isfile(file_path)
    
    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=reviews[0].keys())
        
        if not file_exists:
            writer.writeheader() 
        
        for review in reviews:
            writer.writerow(review)


def get_next_button(driver):
    try:
        return driver.find_element(By.CLASS_NAME, "nav.next.primary")
    except NoSuchElementException:
        return None

def random_sleep(min_sec, max_sec):
    time.sleep(random.uniform(min_sec, max_sec))

def main():
    meta_df = pd.read_csv("./scraped_data.csv")
    batch_size = 1
    processed_links_file = 'processed_links.txt'
    processed_links = read_processed_links(processed_links_file)
    output_file = 'scraped_airline_reviews.csv'

    driver = initialize_webdriver()
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
    )

    for batch in create_batches(meta_df, batch_size):
        for review_link in batch['Review Link']:
            if review_link in processed_links:
                print(f"Skipping already processed link: {review_link}")
                continue

            print(f'Starting to scrape: {review_link}')
            random_sleep(15, 20) 

            driver.get(review_link)
            while True:
                if check_for_captcha(driver):
                    input("CAPTCHA detected. Please solve it and then press Enter to continue...")
                    time.sleep(5)

                # Scrape the current page
                elements_html = scrape_webpage(driver, review_link)
                all_reviews = [review for review in (parse_review(html, review_link) for html in elements_html) if review and (review.get('review_text') or review.get('overall_rating'))]

                if all_reviews:
                    save_reviews_to_csv(all_reviews, output_file)

                # Navigate to the next page
                next_button = get_next_button(driver)
                if next_button and 'disabled' not in next_button.get_attribute('class'):
                    href = next_button.get_attribute('href')
                    print(f"Navigating to: {href}")
                    next_button.click()
                    random_sleep(5, 10)
                else:
                    print("No more pages to navigate or Next button is disabled.")
                    break

            add_processed_link(processed_links_file, review_link)

    driver.quit()

main()

# integrate the cookie consent thing
# batch processing (10 links at a time then break/ip rotate)

started to init
init complete
Starting to scrape: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-Adria-Airways-No-Longer-Operating


CAPTCHA detected. Please solve it and then press Enter to continue... 


started scraping


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.85)
Stacktrace:
0   chromedriver                        0x0000000102f367dc chromedriver + 4040668
1   chromedriver                        0x0000000102f2e9e0 chromedriver + 4008416
2   chromedriver                        0x0000000102ba1870 chromedriver + 284784
3   chromedriver                        0x0000000102b7c064 chromedriver + 131172
4   chromedriver                        0x0000000102c0b2b0 chromedriver + 717488
5   chromedriver                        0x0000000102c1e75c chromedriver + 796508
6   chromedriver                        0x0000000102bd974c chromedriver + 513868
7   chromedriver                        0x0000000102bda044 chromedriver + 516164
8   chromedriver                        0x0000000102efba04 chromedriver + 3799556
9   chromedriver                        0x0000000102effee4 chromedriver + 3817188
10  chromedriver   

##### testing parse codes

In [259]:
txt = ['<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/geneoV" target="_self" class="kjIqZ I ui_social_avatar inline"><img src="https://media-cdn.tripadvisor.com/media/photo-t/0b/00/af/16/geneov.jpg" alt="avatar-image" loading="lazy"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"><div class="overflow "><span class="GXoUa NF S8 _S MVUuZ"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP"><path d="M5 14a2 2 0 100-4 2 2 0 000 4zM19 14a2 2 0 100-4 2 2 0 000 4zM12 14a2 2 0 100-4 2 2 0 000 4z"></path></svg></span></div><span class="_m"></span></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/geneoV">Gene S</a> wrote a review Apr 2019</span></div><div class="MziKN"><span class="RdTWF"><span class="default LXUOn small"><span class="ui_icon map-pin-fill fXexN"></span>Toronto, Canada</span></span><span class="phMBo"><span><span class="yRNgz">447</span> contributions</span></span><span class="phMBo"><span><span class="yRNgz">95</span> helpful votes</span></span></div></div></div><div class="WAllg _T" data-reviewid="670493579"><div class="IkECb f O"><div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_10"></span></div><div class="tNvbV f Ng"><div class="dmRSR n R2 S2">Medellin - Apartado</div><div class="dmRSR n R2 S2">Domestic</div><div class="dmRSR n R2 S2">Economy</div></div></div><div class="KgQgP MC _S b S6 H5 _a" dir="ltr" data-test-target="review-title"><a rel="noopener" href="/ShowUserReviews-g1-d11831129-r670493579-Aerolinea_de_Antioquia-World.html" target="_blank" class="Qwuub"><span><span>Flight Cancelled, "DID NOT GET MY MONEY BACK"</span></span></a></div><div class="vTVDc"><div class="_T FKffI"><div class="fIrGe _T" style="max-height: none; line-break: normal; cursor: auto;"><span class="QewHA H4 _a" data-test-target="review-text"><span>I purchased two tickets from Medellin to Acandi in January 2019 and paid for both on my credit card. In late March I was informed of the cancellation of this flight (in fact the service between the two locations was discontinued). No alternative was offered.<br> I was told to go to the airport office to get a refund, but I had already left Medellin so my wife went in my place. They gave her a refund for my ticket but the other ticket was in a friends name so they had to have his permission although I paid for the ticket with my visa. Through emails, the permissions were given and my wife returned to the airport to get the promised refund only to be told it would be credited back to the visa card within two  weeks however its been more that three weeks now. This is a terrible airline to deal with and they are extremely uncaring about the customer. They keep giving out incorrect information and do not do what they say they will. They are incompetent. Use another airline or take the bus, Aerolinas de Antioquia is unreliable.</span></span></div><div class="lszDU" style="line-height: 20px;"><div class="TnInx" data-test-target="expand-review"><span class="Ignyf _S Z">Read more</span><span class="ui_icon caret-down Lvqmo"></span></div></div><div class="JTbpC" style="line-height: 20px;"><div class="TnInx"><span class="uoxiZ _S Z">Read less</span><span class="ui_icon caret-up Lvqmo"></span></div></div></div><span class="teHYY _R Me S4 H3"><span class="usajM">Date of travel:</span> April 2019</span><div class="ZzICe Me f"><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_10"></span></span><span>Customer service</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_10"></span></span><span>Value for money</span></div></div><div class="XCJRC S2 Me MD i">This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC. Tripadvisor performs checks on reviews.</div><div class="YnxVY"><span class="wpsrP f _T Xd Y"></span><div class="qIICj f _T Me z"><span class="zMxkM f _T M _S"><span class="ui_icon thumbs-up bohqm"></span><span class="ui_link RAPNy">Helpful</span></span><span class="CXSid f"><div><span><div class="mGZLW f _T M _S qNoXj"><span><span class="ui_icon share AWqKr"></span><span class="ui_link duDPx socialCTA">Share </span></span></div></span></div></span></div></div></div></div></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/JorgeOyola" target="_self" class="kjIqZ I ui_social_avatar inline"><img src="https://media-cdn.tripadvisor.com/media/photo-t/18/3d/e8/23/jorgeoyola.jpg" alt="avatar-image" loading="lazy"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"><div class="overflow "><span class="GXoUa NF S8 _S MVUuZ"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP"><path d="M5 14a2 2 0 100-4 2 2 0 000 4zM19 14a2 2 0 100-4 2 2 0 000 4zM12 14a2 2 0 100-4 2 2 0 000 4z"></path></svg></span></div><span class="_m"></span></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/JorgeOyola">JorgeOyola</a> wrote a review Apr 2019</span></div><div class="MziKN"><span class="phMBo"><span><span class="yRNgz">136</span> contributions</span></span><span class="phMBo"><span><span class="yRNgz">87</span> helpful votes</span></span></div></div></div><div class="WAllg _T" data-reviewid="664674928"><div class="IkECb f O"><div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_20"></span></div><div class="tNvbV f Ng"><div class="dmRSR n R2 S2">Medellin - Monteria</div><div class="dmRSR n R2 S2">Domestic</div><div class="dmRSR n R2 S2">Economy</div></div></div><div class="KgQgP MC _S b S6 H5 _a" dir="ltr" data-test-target="review-title"><a rel="noopener" href="/ShowUserReviews-g1-d11831129-r664674928-Aerolinea_de_Antioquia-World.html" target="_blank" class="Qwuub"><span><span>Do not care about you when flights are cancelled</span></span></a></div><div class="vTVDc"><div class="_T FKffI"><div class="fIrGe _T" style="max-height: none; line-break: normal; cursor: auto;"><span class="QewHA H4 _a" data-test-target="review-text"><span>I had used the airline several times. It was ok. I heard some stories about overbooking, I never experienced such problems. One day my flight was cancelled due to weather conditions. I got back part of the money that I paid for the tickets. They claimed they can not refund a tax, so don\'t, but give something else. Since that time I have never used their services anymore.</span></span></div><div class="lszDU" style="line-height: 20px;"><div class="TnInx" data-test-target="expand-review"><span class="Ignyf _S Z">Read more</span><span class="ui_icon caret-down Lvqmo"></span></div></div><div class="JTbpC" style="line-height: 20px;"><div class="TnInx"><span class="uoxiZ _S Z">Read less</span><span class="ui_icon caret-up Lvqmo"></span></div></div></div><span class="teHYY _R Me S4 H3"><span class="usajM">Date of travel:</span> October 2018</span><div class="ZzICe Me f"><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_20"></span></span><span>Legroom</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_10"></span></span><span>Seat comfort</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_10"></span></span><span>In-flight Entertainment</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_10"></span></span><span>Customer service</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_20"></span></span><span>Value for money</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_40"></span></span><span>Cleanliness</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_40"></span></span><span>Check-in and boarding</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_20"></span></span><span>Food and Beverage</span></div></div><div class="XCJRC S2 Me MD i">This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC. Tripadvisor performs checks on reviews.</div><div class="YnxVY"><span class="wpsrP f _T Xd Y"></span><div class="qIICj f _T Me z"><span class="zMxkM f _T M _S"><span class="ui_icon thumbs-up bohqm"></span><span class="ui_link RAPNy">Helpful</span></span><span class="CXSid f"><div><span><div class="mGZLW f _T M _S qNoXj"><span><span class="ui_icon share AWqKr"></span><span class="ui_link duDPx socialCTA">Share </span></span></div></span></div></span></div></div></div></div></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/travelin_tommy" target="_self" class="kjIqZ I ui_social_avatar inline"><img src="https://media-cdn.tripadvisor.com/media/photo-o/08/65/c7/8d/travelin-tommy.jpg" alt="avatar-image" loading="lazy"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"><div class="overflow "><span class="GXoUa NF S8 _S MVUuZ"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP"><path d="M5 14a2 2 0 100-4 2 2 0 000 4zM19 14a2 2 0 100-4 2 2 0 000 4zM12 14a2 2 0 100-4 2 2 0 000 4z"></path></svg></span></div><span class="_m"></span></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/travelin_tommy">travelin_tommy</a> wrote a review Mar 2019</span></div><div class="MziKN"><span class="RdTWF"><span class="default LXUOn small"><span class="ui_icon map-pin-fill fXexN"></span>San Francisco, California</span></span><span class="phMBo"><span><span class="yRNgz">390</span> contributions</span></span><span class="phMBo"><span><span class="yRNgz">212</span> helpful votes</span></span></div></div></div><div class="WAllg _T" data-reviewid="656165517"><div class="IkECb f O"><div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_30"></span></div><div class="tNvbV f Ng"><div class="dmRSR n R2 S2">Pereira - Medellin</div><div class="dmRSR n R2 S2">Domestic</div><div class="dmRSR n R2 S2">Economy</div></div></div><div class="KgQgP MC _S b S6 H5 _a" dir="ltr" data-test-target="review-title"><a rel="noopener" href="/ShowUserReviews-g1-d11831129-r656165517-Aerolinea_de_Antioquia-World.html" target="_blank" class="Qwuub"><span><span>Merely OK</span></span></a></div><div class="vTVDc"><div class="_T FKffI"><div class="fIrGe _T" style="max-height: none; line-break: normal; cursor: auto;"><span class="QewHA H4 _a" data-test-target="review-text"><span>Aerolineas de Antioquia is also known as ADA (when you have to search to find the check-in counter). This brief review is based on a single short flight from Pereira to Medellin. One important item is that our departure and arrival were on time. Secondly, our bags showed up too! Thirdly, the price for the flight was fairly reasonable (well, maybe not Easyjet or Ryanair cheap, but not too bad). Seating space on the plane is tight; at 6 ft., 1 in., I’m not especially tall (nor fat), but my knees were pressed against the seat in front without that passenger reclining. An irritable issue is that on-line check-in is not permitted, so it was necessary to pay a small fee at check-in to get a physical boarding pass. It is also necessary to pay another fee for airport tax, and finally, it was necessary to pay as well because one bag was over the 10-kg limit; it was not permitted to average the weights of my check-in bag and my wife’s check-in bag. Our boarding pass told us to be at the gate at a particular time or, we were told, we might risk losing our seat on the flight. However, security agents would not even let us get into the security line to proceed to the gate until thirty minutes past the time the airline agent (and boarding pass) said we must be at the gate; third attempt to go was acceptable. As I was confident there must be others with this same predicament, I did not panic, but it was still a little disconcerting. Also, it is necessary to be careful once in the gate area, as there are multiple flights leaving from the same gate about the same time and neither the signage at the gate nor announcements clearly indicated when to proceed through the gate and to the plane. I am not sure how one would deal with these things if you do not know Spanish; while not actually fluent, I can converse one-on-one with most people on most topics. Yet, I found it a chaotic experience.</span></span></div><div class="lszDU" style="line-height: 20px;"><div class="TnInx" data-test-target="expand-review"><span class="Ignyf _S Z">Read more</span><span class="ui_icon caret-down Lvqmo"></span></div></div><div class="JTbpC" style="line-height: 20px;"><div class="TnInx"><span class="uoxiZ _S Z">Read less</span><span class="ui_icon caret-up Lvqmo"></span></div></div></div><span class="teHYY _R Me S4 H3"><span class="usajM">Date of travel:</span> February 2019</span><div class="ZzICe Me f"><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_20"></span></span><span>Legroom</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_30"></span></span><span>Seat comfort</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_10"></span></span><span>In-flight Entertainment</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_30"></span></span><span>Customer service</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_30"></span></span><span>Value for money</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_30"></span></span><span>Cleanliness</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_20"></span></span><span>Check-in and boarding</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_10"></span></span><span>Food and Beverage</span></div></div><div class="XCJRC S2 Me MD i">This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC. Tripadvisor performs checks on reviews.</div><div class="YnxVY"><span class="wpsrP f _T Xd Y"></span><div class="qIICj f _T Me z"><span class="zMxkM f _T M _S"><span class="ui_icon thumbs-up bohqm"></span><span class="ui_link RAPNy">Helpful</span></span><span class="CXSid f"><div><span><div class="mGZLW f _T M _S qNoXj"><span><span class="ui_icon share AWqKr"></span><span class="ui_link duDPx socialCTA">Share </span></span></div></span></div></span></div></div></div></div></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/mirandodelmonte" target="_self" class="kjIqZ I ui_social_avatar inline"><img src="https://media-cdn.tripadvisor.com/media/photo-t/04/4f/ac/79/decours.jpg" alt="avatar-image" loading="lazy"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"><div class="overflow "><span class="GXoUa NF S8 _S MVUuZ"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP"><path d="M5 14a2 2 0 100-4 2 2 0 000 4zM19 14a2 2 0 100-4 2 2 0 000 4zM12 14a2 2 0 100-4 2 2 0 000 4z"></path></svg></span></div><span class="_m"></span></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/mirandodelmonte">mirandodelmonte</a> wrote a review Aug 2018</span></div><div class="MziKN"><span class="RdTWF"><span class="default LXUOn small"><span class="ui_icon map-pin-fill fXexN"></span>Berlin, Germany</span></span><span class="phMBo"><span><span class="yRNgz">183</span> contributions</span></span><span class="phMBo"><span><span class="yRNgz">73</span> helpful votes</span></span></div></div></div><div class="pDrIj f z"><div class="MtByY _T PfFfr"><div class="BSBvb GA" style="background-size:cover;height:128px;width:100%;background-image:none"></div></div><div class="MtByY _T PfFfr"><div class="BSBvb GA" style="background-size:cover;height:128px;width:100%;background-image:none"></div></div><div class="MtByY _T PfFfr"><div class="BSBvb GA" style="background-size:cover;height:128px;width:100%;background-image:none"></div></div></div><div class="WAllg _T" data-reviewid="604326844"><div class="IkECb f O"><div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div><div class="tNvbV f Ng"><div class="dmRSR n R2 S2">Medellin - Bahia Solano</div><div class="dmRSR n R2 S2">Domestic</div><div class="dmRSR n R2 S2">Economy</div></div></div><div class="KgQgP MC _S b S6 H5 _a" dir="ltr" data-test-target="review-title"><a rel="noopener" href="/ShowUserReviews-g1-d11831129-r604326844-Aerolinea_de_Antioquia-World.html" target="_blank" class="Qwuub"><span><span>small but reliable airline</span></span></a></div><div class="vTVDc"><div class="_T FKffI"><div class="fIrGe _T" style="max-height: none; line-break: normal; cursor: auto;"><span class="QewHA H4 _a" data-test-target="review-text"><span>small, local airline that offers a few local flights out of Medellin. Had a roundtrip to Bahia Solano (Choco). Everything was fine and on time.</span></span></div><div class="lszDU" style="line-height: 20px;"><div class="TnInx" data-test-target="expand-review"><span class="Ignyf _S Z">Read more</span><span class="ui_icon caret-down Lvqmo"></span></div></div><div class="JTbpC" style="line-height: 20px;"><div class="TnInx"><span class="uoxiZ _S Z">Read less</span><span class="ui_icon caret-up Lvqmo"></span></div></div></div><span class="teHYY _R Me S4 H3"><span class="usajM">Date of travel:</span> August 2018</span><div class="ZzICe Me f"><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_40"></span></span><span>Legroom</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_40"></span></span><span>Seat comfort</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_50"></span></span><span>Customer service</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_50"></span></span><span>Value for money</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_50"></span></span><span>Cleanliness</span></div><div class="hemdC S2 H2"><span class="Nd"><span class="ui_bubble_rating bubble_20"></span></span><span>Check-in and boarding</span></div></div><div class="XCJRC S2 Me MD i">This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC. Tripadvisor performs checks on reviews.</div><div class="YnxVY"><span class="wpsrP f _T Xd Y"></span><div class="qIICj f _T Me z"><span class="zMxkM f _T M _S"><span class="ui_icon thumbs-up bohqm"></span><span class="ui_link RAPNy">Helpful</span></span><span class="CXSid f"><div><span><div class="mGZLW f _T M _S qNoXj"><span><span class="ui_icon share AWqKr"></span><span class="ui_link duDPx socialCTA">Share </span></span></div></span></div></span></div></div></div></div></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/R3liableCustomer" target="_self" class="kjIqZ I ui_social_avatar inline"><img src="https://media-cdn.tripadvisor.com/media/photo-t/1a/f6/ed/00/default-avatar-2020-4.jpg" alt="avatar-image" loading="lazy"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"><div class="overflow "><span class="GXoUa NF S8 _S MVUuZ"><svg viewBox="0 0 24 24" width="1em" height="1em" class="d Vb UmNoP"><path d="M5 14a2 2 0 100-4 2 2 0 000 4zM19 14a2 2 0 100-4 2 2 0 000 4zM12 14a2 2 0 100-4 2 2 0 000 4z"></path></svg></span></div><span class="_m"></span></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/R3liableCustomer">R3liableCustomer</a> wrote a review Jun 2017</span></div><div class="MziKN"><span class="RdTWF"><span class="default LXUOn small"><span class="ui_icon map-pin-fill fXexN"></span>Miami, Florida</span></span><span class="phMBo"><span><span class="yRNgz">2</span> contributions</span></span><span class="phMBo"><span><span class="yRNgz">6</span> helpful votes</span></span></div></div></div><div class="WAllg _T" data-reviewid="495164829"><div class="IkECb f O"><div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_20"></span></div><div class="tNvbV f Ng"><div class="dmRSR n R2 S2">Monteria - Cartagena</div><div class="dmRSR n R2 S2">Domestic</div><div class="dmRSR n R2 S2">Economy</div></div></div><div class="KgQgP MC _S b S6 H5 _a" dir="ltr" data-test-target="review-title"><a rel="noopener" href="/ShowUserReviews-g1-d11831129-r495164829-Aerolinea_de_Antioquia-World.html" target="_blank" class="Qwuub"><span><span>Be Aware!! --&gt; Delayed Flights</span></span></a></div><div class="vTVDc"><div class="_T FKffI"><div class="fIrGe _T" style="max-height: none; line-break: normal; cursor: auto;"><span class="QewHA H4 _a" data-test-target="review-text"><span>If you want to get to your destination ON TIME.. DON\'T flight with ADA... Seriously. Usually their flights are delayed (3-4hours). I\'m sure you can find better options. Terrible customer service as well.</span></span></div><div class="lszDU" style="line-height: 20px;"><div class="TnInx" data-test-target="expand-review"><span class="Ignyf _S Z">Read more</span><span class="ui_icon caret-down Lvqmo"></span></div></div><div class="JTbpC" style="line-height: 20px;"><div class="TnInx"><span class="uoxiZ _S Z">Read less</span><span class="ui_icon caret-up Lvqmo"></span></div></div></div><span class="teHYY _R Me S4 H3"><span class="usajM">Date of travel:</span> April 2017</span><div class="XCJRC S2 Me MD i">This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC. Tripadvisor performs checks on reviews.</div><div class="YnxVY"><span class="wpsrP f _T Xd Y"></span><div class="qIICj f _T Me z"><span class="zMxkM f _T M _S"><span class="ui_icon thumbs-up bohqm"></span><span class="ui_link RAPNy">Helpful</span></span><span class="CXSid f"><div><span><div class="mGZLW f _T M _S qNoXj"><span><span class="ui_icon share AWqKr"></span><span class="ui_link duDPx socialCTA">Share </span></span></div></span></div></span></div></div></div></div></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/mirandodelmonte" target="_self" class="kjIqZ I ui_social_avatar inline"><img alt="avatar-image"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"></div><div class="cRVSd"><span></span></div><div class="MziKN"></div></div></div><div class="oqErg"><div class="MBufR"><div class="BSBvb GA" style="background-size:cover;height:300px;width:100%;background-image:none"></div></div></div><span class="wpsrP f _T Xd Y"></span></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/mirandodelmonte" target="_self" class="kjIqZ I ui_social_avatar inline"><img alt="avatar-image"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"></div><div class="cRVSd"><span></span></div><div class="MziKN"></div></div></div><div class="oqErg"><div class="MBufR"><div class="BSBvb GA" style="background-size:cover;height:300px;width:100%;background-image:none"></div></div></div><span class="wpsrP f _T Xd Y"></span></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/mirandodelmonte" target="_self" class="kjIqZ I ui_social_avatar inline"><img alt="avatar-image"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"></div><div class="cRVSd"><span></span></div><div class="MziKN"></div></div></div><div class="oqErg"><div class="MBufR"><div class="BSBvb GA" style="background-size:cover;height:300px;width:100%;background-image:none"></div></div></div><span class="wpsrP f _T Xd Y"></span></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/geneoV" target="_self" class="kjIqZ I ui_social_avatar inline"><img alt="avatar-image"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/geneoV">Gene S</a> wrote a tip Apr 2019</span></div><div class="MziKN"></div></div></div><a href="/ShowUserReviews-g1-d11831129-r670493579-Aerolinea_de_Antioquia-World.html" class="taDzF _R" target="_blank"><span class="ui_bubble_rating bubble_10"></span></a><blockquote data="tip_text">"<span>Find another way</span>"</blockquote><div><a href="/ShowUserReviews-g1-d11831129-r670493579-Aerolinea_de_Antioquia-World.html" class="UGlEC _R b" target="_blank" data="tip_link">Read full review</a></div></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/travelin_tommy" target="_self" class="kjIqZ I ui_social_avatar inline"><img alt="avatar-image"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/travelin_tommy">travelin_tommy</a> wrote a tip Mar 2019</span></div><div class="MziKN"></div></div></div><a href="/ShowUserReviews-g1-d11831129-r656165517-Aerolinea_de_Antioquia-World.html" class="taDzF _R" target="_blank"><span class="ui_bubble_rating bubble_30"></span></a><blockquote data="tip_text">"<span>If you can identify an agent of the airline, let them know you\'re there periodically.</span>"</blockquote><div><a href="/ShowUserReviews-g1-d11831129-r656165517-Aerolinea_de_Antioquia-World.html" class="UGlEC _R b" target="_blank" data="tip_link">Read full review</a></div></div>', '<div class="lgfjP Gi z pBVnE MD bZHZM"><div class="sCZGP"><a href="/Profile/R3liableCustomer" target="_self" class="kjIqZ I ui_social_avatar inline"><img alt="avatar-image"></a><div class="IHLTq _Z o"><div class="jXhbq f _Y _m"></div><div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/R3liableCustomer">R3liableCustomer</a> wrote a tip Jun 2017</span></div><div class="MziKN"></div></div></div><a href="/ShowUserReviews-g1-d11831129-r495164829-Aerolinea_de_Antioquia-World.html" class="taDzF _R" target="_blank"><span class="ui_bubble_rating bubble_20"></span></a><blockquote data="tip_text">"<span>Try to find another Airline! ADA isn\'t a good option!</span>"</blockquote><div><a href="/ShowUserReviews-g1-d11831129-r495164829-Aerolinea_de_Antioquia-World.html" class="UGlEC _R b" target="_blank" data="tip_link">Read full review</a></div></div>']

from bs4 import BeautifulSoup
import re
from datetime import datetime

existing_reviews = set()

def parse_review(html, url):
    soup = BeautifulSoup(html, 'html.parser')
    review_data = {}
    
    review_data['airline_url'] = url
    user_info = soup.find('a', class_='ui_header_link')
    if user_info:
        user_name = user_info.get_text(strip=True)
        review_date_div = soup.find('div', class_='cRVSd')
        if review_date_div:
            review_date = review_date_div.get_text()
            unique_id = (url, user_name, review_date)
            if unique_id in existing_reviews:
                return None
            existing_reviews.add(unique_id)
            review_data['user_name'] = user_name
            review_data['review_date'] = review_date
            review_data['scraped_date'] = datetime.today().strftime('%Y-%m-%d')
        else:
            return None
    else:
        return None

    travel_details = soup.find_all('div', class_='dmRSR n R2 S2')
    review_data['travel_details'] = [detail.get_text(strip=True) for detail in travel_details]

    review_title = soup.find('div', class_='KgQgP MC _S b S6 H5 _a')
    review_data['review_title'] = review_title.get_text(strip=True) if review_title else None
    review_text = soup.find('span', class_='QewHA H4 _a')
    review_data['review_text'] = review_text.get_text(strip=True) if review_text else None
    
    overall_rating = soup.find('span', class_='ui_bubble_rating')
    review_data['overall_rating'] = int(overall_rating['class'][1].split('_')[-1]) // 10 if overall_rating else None

    subratings = soup.find_all('div', class_='hemdC S2 H2')
    review_data['subratings'] = {}
    for subrating in subratings:
        category = subrating.get_text(strip=True)
        rating_class = subrating.find('span', class_='ui_bubble_rating')['class'][1]
        review_data['subratings'][category] = int(rating_class.split('_')[-1]) // 10
        
    return review_data

res = []
for i in txt:
    parsed_data = parse_review(i, "example.com")
    if parsed_data is not None:
        res.append(parsed_data)
        print(parsed_data)

df = pd.DataFrame(res)
df

{'airline_url': 'example.com', 'user_name': 'Gene S', 'review_date': 'Gene S wrote a review Apr 2019', 'scraped_date': '2024-01-24', 'travel_details': ['Medellin - Apartado', 'Domestic', 'Economy'], 'review_title': 'Flight Cancelled, "DID NOT GET MY MONEY BACK"', 'review_text': 'I purchased two tickets from Medellin to Acandi in January 2019 and paid for both on my credit card. In late March I was informed of the cancellation of this flight (in fact the service between the two locations was discontinued). No alternative was offered.I was told to go to the airport office to get a refund, but I had already left Medellin so my wife went in my place. They gave her a refund for my ticket but the other ticket was in a friends name so they had to have his permission although I paid for the ticket with my visa. Through emails, the permissions were given and my wife returned to the airport to get the promised refund only to be told it would be credited back to the visa card within two  weeks ho

,airline_url,user_name,review_date,scraped_date,travel_details,review_title,review_text,overall_rating,subratings
0,example.com,Gene S,Gene S wrote a review Apr 2019,2024-01-24,"[Medellin - Apartado, Domestic, Economy]","Flight Cancelled, ""DID NOT GET MY MONEY BACK""",I purchased two tickets from Medellin to Acand...,1,"{'Customer service': 1, 'Value for money': 1}"
1,example.com,JorgeOyola,JorgeOyola wrote a review Apr 2019,2024-01-24,"[Medellin - Monteria, Domestic, Economy]",Do not care about you when flights are cancelled,I had used the airline several times. It was o...,2,"{'Legroom': 2, 'Seat comfort': 1, 'In-flight E..."
2,example.com,travelin_tommy,travelin_tommy wrote a review Mar 2019,2024-01-24,"[Pereira - Medellin, Domestic, Economy]",Merely OK,Aerolineas de Antioquia is also known as ADA (...,3,"{'Legroom': 2, 'Seat comfort': 3, 'In-flight E..."
3,example.com,mirandodelmonte,mirandodelmonte wrote a review Aug 2018,2024-01-24,"[Medellin - Bahia Solano, Domestic, Economy]",small but reliable airline,"small, local airline that offers a few local f...",5,"{'Legroom': 4, 'Seat comfort': 4, 'Customer se..."
4,example.com,R3liableCustomer,R3liableCustomer wrote a review Jun 2017,2024-01-24,"[Monteria - Cartagena, Domestic, Economy]",Be Aware!! --> Delayed Flights,If you want to get to your destination ON TIME...,2,{}
5,example.com,Gene S,Gene S wrote a tip Apr 2019,2024-01-24,[],None,None,1,{}
6,example.com,travelin_tommy,travelin_tommy wrote a tip Mar 2019,2024-01-24,[],None,None,3,{}
7,example.com,R3liableCustomer,R3liableCustomer wrote a tip Jun 2017,2024-01-24,[],None,None,2,{}


##### testing next page

In [283]:
def get_next_button(driver):
    try:
        return driver.find_element(By.CLASS_NAME, "nav.next.primary")
    except NoSuchElementException:
        return None

driver = initialize_webdriver()

try:
    time.sleep(20)
    driver.get("https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or5-Adria-Airways-No-Longer-Operating.html#REVIEWS")

    while True:
        next_button = get_next_button(driver)
        if next_button and 'disabled' not in next_button.get_attribute('class'):
            href = next_button.get_attribute('href')
            print(f"Navigating to: {href}")
            next_button.click()
        else:
            print("No more pages to navigate or Next button is disabled.")
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

started to init
init complete
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or10-Adria-Airways-No-Longer-Operating.html
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or15-Adria-Airways-No-Longer-Operating.html
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or20-Adria-Airways-No-Longer-Operating.html
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or25-Adria-Airways-No-Longer-Operating.html
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or30-Adria-Airways-No-Longer-Operating.html
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or35-Adria-Airways-No-Longer-Operating.html
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or40-Adria-Airways-No-Longer-Operating.html
Navigating to: https://www.tripadvisor.com/Airline_Review-d8728984-Reviews-or45-Adria-Airways-No-Longer-Operating.html
Navigating to: htt